## Visualize Chatbot/Response Data to Inform Business Decisions
#### Emily Flores: Week 2-7 project deliverables portfolio

Work on Customer Support Twitter dataset, modern corpus of tweets and replies. Check, analyze and understand the pattern and conversations, and for study of modern customer support practices and impact. Using the data, analyze and build interactive visualization to make business decision. 


####  Exploratory data analysis 

In [ ]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from collections import Counter 
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from spellchecker import SpellChecker 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from textblob import TextBlob
import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
file_path = 'twcs.csv'
dataset = pd.read_csv('twcs.csv')

In [ ]:
dataset.head()

In [ ]:
rows, columns = dataset.shape

In [ ]:
print("Number of Rows:", rows)

In [ ]:
print("Number of Columns:", columns)

In [ ]:
dataset.info()

In [ ]:
description = dataset.describe()

In [ ]:
print(description)

In [ ]:
dataset.isnull().sum()

In [ ]:
duplicateValues = dataset.duplicated()

In [ ]:
print(duplicateValues.sum())

In [ ]:
dataset[duplicateValues]

#### The data can be cleaned by reordering the 'created_at' attribute by date added.
#### Most recent tweet at the top of data set * oldest at the end.

In [ ]:
dataset.sort_values(by='created_at', ascending=True, inplace=True)

In [ ]:
dataset.head()

#### The data can be cleaned by reorganizing the author_id by alphabetic order.
#### Start with 'author_id' that starts with 'a' & ends with 'z'

In [ ]:
dataset.sort_values(by='author_id', ascending=True, inplace=True)

In [ ]:
dataset.head()

#### I noticed there are repeated 'author_id' we can group them together. 
#### While maintaing the 'author_id' in alphabetic order.
#### This will make it easier to access tweets from specific users. 

In [ ]:
grouped_data = dataset.groupby('author_id')

In [ ]:
dataset.head()

In [ ]:
inbound_count = dataset[dataset['inbound'] == True]['inbound'].count()

In [ ]:
print("Number of inbound responses:", inbound_count)

In [ ]:
outbound_count = dataset[dataset['inbound'] == False]['inbound'].count()

In [ ]:
print("Number of outbound responses:", outbound_count)

In [ ]:
unique_tweets_count = dataset['tweet_id'].nunique()

In [ ]:
print("Number of unique tweets:", unique_tweets_count)

In [ ]:
top_20_users = dataset['author_id'].value_counts().head(20)

In [ ]:
print("Top 20 Twitter users:\n", top_20_users)

### Data Cleaning

In [ ]:
dataset.head()

In [ ]:
userTweetCount = dataset.groupby('author_id').size().reset_index(name='tweetCount')

In [ ]:
topUsers = userTweetCount.sort_values(by='tweetCount', ascending=False)

In [ ]:
print(topUsers.head())

In [ ]:
authorResCount = dataset.groupby('author_id')['response_tweet_id'].count().reset_index(name='response_count')

In [ ]:
topAuthorsRes = authorResCount.sort_values(by='response_count', ascending=False)

In [ ]:
print(topAuthorsRes.head())

In [ ]:
dataset['created_at'] = pd.to_datetime(dataset['created_at'])

In [ ]:
tweetsByDate = dataset.groupby(dataset['created_at'].dt.date).size().reset_index(name='tweet_count')

In [ ]:
print(tweetsByDate)

### Data Cleaning Part 2

In [ ]:
dataset.head(10)

In [ ]:
# Lower casing
dataset['text_lower'] = dataset['text'].str.lower()

In [ ]:
print(dataset)

In [ ]:
# Removal of punctuation 
dataset['no_punct'] = dataset['text_lower'].apply(lambda text: text.translate(str.maketrans('', '', string.punctuation)))

In [ ]:
print(dataset)

#### Stemming (reduce words to their base form)

In [ ]:
stemmer = PorterStemmer()
dataset['text'] = dataset['text'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))

#### Lemmatization (combines similar meaning words as one word)

In [ ]:
lemmatizer = WordNetLemmatizer()
dataset['text'] = dataset['text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

In [ ]:
print(data_set.head(10))

#### Removal of Emoticons (punctuation marks, letters, and numbers)

In [ ]:
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)
dataset['text'] = dataset['text'].apply(remove_emoticons)

#### Remove URLs

In [ ]:
dataset['text'] = dataset['text'].str.replace(r'http\S+|www\S+|https\S+', '', case=False)

#### Remove HTML Tags

In [ ]:
dataset['text'] = dataset['text'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())

#### Spelling Corrector 

In [ ]:
spell = SpellChecker()
dataset['text'] = dataset['text'].apply(lambda x: ' '.join([spell.correction(word) for word in x.split()]))

### Data Analysis 

#### Telecommunication data analysis

What are the most common issues customer are facing based on their tweets to telecommunication companies?

How does response time correlate to customer satisfaction? 

Telecommunication companies such as VerizonSupport, Ask_Spectrum, sprintcare, ect.

In [ ]:
data_set.head(50)

In [ ]:
# Handle missing values
data_set.dropna(subset=['text'], inplace=True)

In [ ]:
# Convert 'created_at' to datetime
data_set['created_at'] = pd.to_datetime(data_set['created_at'])

In [ ]:
# Explore distribution of inbound and outbound tweets
inbound_outbound_counts = data_set['inbound'].value_counts()

In [ ]:
# Plot bar chart
plt.bar(inbound_outbound_counts.keys(), inbound_outbound_counts.values, color=['pink', 'green'], edgecolor='black') 
plt.xlabel('Tweet Type')
plt.ylabel('Count')
plt.title('Inbound vs. Outbound Tweets Distribution')
plt.show()

#### Sentiment analysis 

In [ ]:
def sentiment_analysis(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

In [ ]:
data_set['sentiment'] = data_set['text'].apply(sentiment_analysis)

# Count the occurrences of each sentiment category
sentiment_counts = data_set['sentiment'].value_counts()

# Plotting the bar chart
plt.figure(figsize=(8, 6))
plt.bar(sentiment_counts.index, sentiment_counts.values, color=['green', 'pink', 'red'])
plt.xlabel('Sentiment')
plt.ylabel('Count')
plt.title('Sentiment Analysis Results')
plt.show()

#### Airline data analysis                       

How does response time correlate to customer satisfaction when comparing American Airlines & Delta? 

American Airlines and Delta 

In [ ]:
# Handle missing values
data_set['response_tweet_id'].fillna(-1, inplace=True)
data_set['in_response_to_tweet_id'].fillna(-1, inplace=True)

In [ ]:
data_set

In [ ]:
filtered_data = data_set[data_set['author_id'].str.contains('delta|american', case=False)]

In [ ]:
filtered_data

In [ ]:
# Remove duplicates in the filtered data 
filtered_data = filtered_data.drop_duplicates().copy()

In [ ]:
filtered_data

In [ ]:
# Convert 'response_tweet_id' and 'in_response_to_tweet_id' to numeric 
filtered_data['response_tweet_id'] = pd.to_numeric(filtered_data['response_tweet_id'], errors='coerce')
filtered_data['in_response_to_tweet_id'] = pd.to_numeric(filtered_data['in_response_to_tweet_id'], errors='coerce')

In [ ]:
print(filtered_data)

In [ ]:
# Calculate response time
filtered_data['response_time'] = filtered_data['response_tweet_id'] - filtered_data['in_response_to_tweet_id']

In [ ]:
print(filtered_data)

In [ ]:
# Calculate average response time for each airline
average_response_time = filtered_data.groupby('author_id')['response_time'].mean()

In [ ]:
# Visualize the data
plt.figure(figsize=(8, 6))
colors = ['red', 'blue'] 
average_response_time.plot(kind='bar', color=colors, edgecolor='black') 
plt.title('Average Response Time for American Airlines and Delta')
plt.xlabel('Airline')
plt.ylabel('Average Response Time')
plt.xticks(range(len(average_response_time.index)), average_response_time.index, rotation=0)
plt.ylim(0)  
plt.tight_layout() 

#### Statistical Prediction/modeling

In [ ]:
correlation = filtered_data.corr(numeric_only=True)

In [ ]:
filtered_data['response_time'] = filtered_data['response_tweet_id'] - filtered_data['in_response_to_tweet_id']

average_response_time = filtered_data.groupby('author_id')['response_time'].mean()

print(average_response_time)

#### Convert to postive for better analysis of results

In [ ]:
filtered_data['response_time'] = filtered_data['response_tweet_id'] - filtered_data['in_response_to_tweet_id']

average_response_time = filtered_data.groupby('author_id')['response_time'].mean().abs()

print(average_response_time)

Results 

American Airlines with an average response time of approximately 983,238.5 units is faster than Delta with an average response time of approximately 1,069,096 units. This can correlate to customers being more satified with American Airlines customer support via the tweet data set. 

### Data Visualizing (Technology & Airline Sectors) 

#### Technology Data Analysis

#### Step 1. 
What is the relationship between average response time & customer satisfaction for each technology support service?

#### Step 2. 
MicrosoftHelps, AppleSupport, DellCares, HPSupport

In [ ]:
filtered_data = dataset[dataset['author_id'].str.contains('MicrosoftHelps|AppleSupport|DellCares|HPSupport', case=False)]

#### Display filtered data

In [ ]:
filtered_data = dataset[dataset['author_id'].str.contains('MicrosoftHelps|AppleSupport|DellCares|HPSupport', case=False)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(filtered_data.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[filtered_data.tweet_id, filtered_data.author_id, filtered_data.inbound, filtered_data.created_at, filtered_data.text, filtered_data.response_tweet_id, filtered_data.in_response_to_tweet_id],
               fill_color='lavender',
               align='left'))
])

fig.show()

In [ ]:
# Handle Missing Values
cleaned_data = dataset.dropna()

In [ ]:
# Convert to DateTime
dataset['created_at'] = pd.to_datetime(dataset['created_at'])

In [ ]:
# Group data by 'author_id' and 'inbound'
grouped_data = filtered_data.groupby(['author_id', 'inbound']).size().reset_index(name='count')

#### Explore & Visulaize distribution of inbound and outbound tweets 

In [ ]:
fig = px.bar(grouped_data, x='author_id', y='count', color='inbound',
             labels={'count': 'Number of Tweets', 'inbound': 'Tweet Type'},
             title='Distribution of Inbound and Outbound Tweets for Companies',
             category_orders={'author_id': ['MicrosoftHelps', 'AppleSupport', 'DellCares', 'HPSupport']})

fig.show()

#### Calculating the average response time for each company 

In [ ]:
filtered_data = dataset[dataset['author_id'].str.contains('MicrosoftHelps|AppleSupport|DellCares|HPSupport', case=False)]

#### Convert 'created_at' and 'response_tweet_id' columns to datetime

In [ ]:
filtered_data['created_at'] = pd.to_datetime(filtered_data['created_at'], errors='coerce').dt.tz_localize(None)
filtered_data['response_tweet_id'] = pd.to_datetime(filtered_data['response_tweet_id'], errors='coerce').dt.tz_localize(None)

#### Filter out rows where 'response_tweet_id' or 'created_at' couldn't be parsed

In [ ]:
cleaned_filtered_data = filtered_data.dropna(subset=['created_at', 'response_tweet_id'])

#### Calculate response time in seconds 

In [ ]:
cleaned_filtered_data['response_time'] = (cleaned_filtered_data['response_tweet_id'] - cleaned_filtered_data['created_at']).dt.total_seconds()

#### Results for average reponse times

In [ ]:
average_response_time = cleaned_filtered_data.groupby('author_id')['response_time'].mean()
print(average_response_time)

#### Visualized the Average Response Time for each Technology Company

In [ ]:
data = {
    'AppleSupport': 1.145536e+08,
    'DellCares': 1.839929e+07,
    'HPSupport': 7.365916e+08,
    'MicrosoftHelps': 3.703211e+08
}

author_ids = list(data.keys())
response_times = list(data.values())

plt.figure(figsize=(10, 6))
plt.bar(author_ids, response_times, color='skyblue')
plt.xlabel('Author ID')
plt.ylabel('Average Response Time')
plt.title('Average Response Times for companies')
plt.xticks(rotation=45)  
plt.tight_layout()
plt.show()


Results

Apple support = most tweets & Dell care = least tweets

Dell cares = least time to response & HP support = most time to response 

### Airline Data Analysis

##### Step 1.  
How does response time correlate to customer satisfaction when comparing American Airlines & Delta?

#### Step 2.
American Airlines and Delta

In [ ]:
# Handle missing values
dataset['response_tweet_id'].fillna(-1, inplace=True)
dataset['in_response_to_tweet_id'].fillna(-1, inplace=True)

In [ ]:
filtered_data = dataset[dataset['author_id'].str.contains('delta|american', case=False)]

In [ ]:
filtered_data = dataset[dataset['author_id'].str.contains('delta|american', case=False)]

fig = go.Figure(data=[go.Table(
    header=dict(values=list(filtered_data.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[filtered_data.tweet_id, filtered_data.author_id, filtered_data.inbound, filtered_data.created_at, filtered_data.text, filtered_data.response_tweet_id, filtered_data.in_response_to_tweet_id],
               fill_color='lavender',
               align='left'))
])
fig.show()

In [ ]:
# Remove duplicates in the filtered data 
filtered_data = filtered_data.drop_duplicates().copy()

In [ ]:
# Convert 'response_tweet_id' and 'in_response_to_tweet_id' to numeric 
filtered_data['response_tweet_id'] = pd.to_numeric(filtered_data['response_tweet_id'], errors='coerce')
filtered_data['in_response_to_tweet_id'] = pd.to_numeric(filtered_data['in_response_to_tweet_id'], errors='coerce')

In [ ]:
# Calculate response time
filtered_data['response_time'] = filtered_data['response_tweet_id'] - filtered_data['in_response_to_tweet_id']

In [ ]:
# Calculate average response time for each airline
average_response_time = filtered_data.groupby('author_id')['response_time'].mean()

#### Statistical Predicitions / Modeling

In [ ]:
correlation = filtered_data.corr(numeric_only=True)

filtered_data['response_time'] = filtered_data['response_tweet_id'] - filtered_data['in_response_to_tweet_id']

average_response_time = filtered_data.groupby('author_id')['response_time'].mean()

print(average_response_time)

In [ ]:
filtered_data['response_time'] = filtered_data['response_tweet_id'] - filtered_data['in_response_to_tweet_id']

average_response_time = filtered_data.groupby('author_id')['response_time'].mean().abs()

print(average_response_time)

In [ ]:
colors = ['red', 'blue'] 
average_response_time.plot(kind='bar', color=colors, edgecolor='black')
plt.xlabel('Author ID')
plt.ylabel('Average Response Time')
plt.title('Average Response Time by Author ID')
plt.tight_layout()
plt.show()

Result

American Airlines with an average response time of approximately 983,238.5 units is faster than Delta with an average response time of approximately 1,069,096 units. This can correlate to customers being more satified with American Airlines customer support via the tweet data set